Etiquetador con autodistill

Librerias

In [1]:
import os
import cv2
from autodistill_grounded_sam import GroundedSAM
from autodistill.detection import CaptionOntology
from autodistill_yolov8 import YOLOv8
from autodistill_grounding_dino import GroundingDINO
#from autodistill_yolov11 import YOLOv11
import torch  # <-- 1. Asegúrate de importar torch

torch.use_deterministic_algorithms(False)

Definicion de directorios

In [2]:
# --- 1. Configuración Inicial ---
VIDEO_PATH = "videos/C05_CRUCE_COLON_BAJO_PIM_CRR_20250627-073209.mp4"
INPUT_IMAGE_DIR = "CRUCE_COLON_1_class"
OUTPUT_DATASET_DIR = "dataset_cruce_3"

# Crear directorios si no existen
os.makedirs(INPUT_IMAGE_DIR, exist_ok=True)
os.makedirs(OUTPUT_DATASET_DIR, exist_ok=True)


Extractor de fotogramas

In [3]:
# --- 2. Extracción de Fotogramas del Video ---
print("Extrayendo fotogramas del video...")
cap = cv2.VideoCapture(VIDEO_PATH)
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Guardar un fotograma cada segundo (puedes ajustar este valor)
    if frame_count % int(cap.get(cv2.CAP_PROP_FPS)) == 0:
        cv2.imwrite(os.path.join(INPUT_IMAGE_DIR, f"frame_{frame_count}.jpg"), frame)
    frame_count += 1
cap.release()
print(f"Se extrajeron {len(os.listdir(INPUT_IMAGE_DIR))} fotogramas.")

Extrayendo fotogramas del video...
Se extrajeron 509 fotogramas.


Definir Ontologia

In [ ]:
# --- 3. Definición de la Ontología ---
# Aquí defines qué objetos quieres detectar y cómo se llamarán las clases.
# El formato es { "lo que el modelo base debe buscar": "el nombre de tu clase" }

ontology = CaptionOntology({
    "camión, excavadora, grúa, automovil, camioneta,autobus, , minivan, microbus, Bus, truck, crane, pick up, vehicle,": "vehícule",
})


Etiquetado (Autodistill)

In [5]:

# --- 4. Etiquetado Automático con el Modelo Base ---
print("Iniciando el etiquetado automático...")
# Usaremos GroundedSAM como nuestro modelo base
base_model = GroundingDINO(ontology=ontology)

# Etiquetar la carpeta de imágenes extraídas
# Esto creará un nuevo directorio con las imágenes y sus archivos de etiquetas en formato YOLO
base_model.label(
    input_folder=INPUT_IMAGE_DIR,
    output_folder=OUTPUT_DATASET_DIR,
)
print("Etiquetado automático completado.")
print(f"Los datos etiquetados se han guardado en: {OUTPUT_DATASET_DIR}")



Iniciando el etiquetado automático...
trying to load grounding dino directly


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3596.)


final text_encoder_type: bert-base-uncased


Labeling CRUCE_COLON_1_class\frame_0.jpg:   0%|          | 0/509 [00:00<?, ?it/s]The `device` argument is deprecated and will be removed in v5 of Transformers.
torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
None of the inputs have requires_grad=True. Gradients will be None
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
Labeling CRUCE_COLON_1_class\frame_990.jpg: 100%|██████████| 509/509 [02:37<00:00,  3.22it/s] 


Labeled dataset created - ready for distillation.
Etiquetado automático completado.
Los datos etiquetados se han guardado en: dataset_cruce_3


Entrenar Modelo (ojala primero revisar en el cvat)

In [31]:
# --- 5. Entrenamiento del Modelo Objetivo ---
print("Iniciando el entrenamiento del modelo objetivo...")
# Usaremos YOLOv8 como nuestro modelo objetivo.
# "yolov8n.pt" es la versión más pequeña y rápida de YOLOv8.
target_model = YOLOv8("yolov8n.pt")

# Entrenar el modelo con los datos que acabamos de etiquetar.
# Autodistill crea un archivo data.yaml dentro del directorio de salida,
# que es lo que necesitamos para el entrenamiento.
target_model.train(
    os.path.join(OUTPUT_DATASET_DIR, "data.yaml"),
    epochs=300 # Puedes ajustar el número de épocas
)
print("¡Entrenamiento completado!")
# El modelo entrenado se guardará en la carpeta 'runs/detect/train'
# (o similar, dependiendo de la versión de ultralytics)

Iniciando el entrenamiento del modelo objetivo...


New https://pypi.org/project/ultralytics/8.3.161 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.81  Python-3.10.18 torch-2.5.1 CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset_cruce_2\data.yaml, epochs=300, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=

¡Entrenamiento completado!


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
